In [4]:
from utils import demix, get_model_from_config
import json

model_type = 'mel_band_roformer'
config_path = 'configs/config_vocals_mel_band_roformer.yaml'
model, config = get_model_from_config(model_type, config_path)

print(json.dumps(list(model.state_dict().keys())[:40], indent=4))

[
    "layers.0.0.layers.0.0.rotary_embed.freqs",
    "layers.0.0.layers.0.0.norm.gamma",
    "layers.0.0.layers.0.0.to_qkv.weight",
    "layers.0.0.layers.0.0.to_gates.weight",
    "layers.0.0.layers.0.0.to_gates.bias",
    "layers.0.0.layers.0.0.to_out.0.weight",
    "layers.0.0.layers.0.1.net.0.gamma",
    "layers.0.0.layers.0.1.net.1.weight",
    "layers.0.0.layers.0.1.net.1.bias",
    "layers.0.0.layers.0.1.net.4.weight",
    "layers.0.0.layers.0.1.net.4.bias",
    "layers.0.0.norm.gamma",
    "layers.0.1.layers.0.0.rotary_embed.freqs",
    "layers.0.1.layers.0.0.norm.gamma",
    "layers.0.1.layers.0.0.to_qkv.weight",
    "layers.0.1.layers.0.0.to_gates.weight",
    "layers.0.1.layers.0.0.to_gates.bias",
    "layers.0.1.layers.0.0.to_out.0.weight",
    "layers.0.1.layers.0.1.net.0.gamma",
    "layers.0.1.layers.0.1.net.1.weight",
    "layers.0.1.layers.0.1.net.1.bias",
    "layers.0.1.layers.0.1.net.4.weight",
    "layers.0.1.layers.0.1.net.4.bias",
    "layers.0.1.norm.gamma",
  